<a href="https://colab.research.google.com/github/onurburakozdemir/FIZ437E/blob/main/HW2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.autograd import Variable
import torch
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
device = "cpu"

Modelleri tanımlıyoruz

In [ ]:
class Logistic_Reg_Model(torch.nn.Module):
  def __init__(self, n_features):
    super(Logistic_Reg_Model, self).__init__()
    self.layer1 = torch.nn.Linear(n_features,10)

  def forward(self, x):
    y_pre = torch.sigmoid(self.layer1(x))
    return y_pre
class SVM_Model(torch.nn.Module):
  def __init__(self, n_features):
    super(SVM_Model, self).__init__()
    self.layer1 = torch.nn.Linear(n_features,10)

  def forward(self, x):
    y_pre = self.layer1(x)
    return y_pre

Eğitmek için fonksiyon tanımlıyoruz

In [ ]:
def trainer(model, criterion, optimizer, test_data, train_data, epoch=10, is_SVM = False):
  acc_arry = []
  for i in range(10):
    print(i, end=' ')
    for j, (data, target) in enumerate(train_data):
      data_size = data.size()
      images = Variable(data.view(-1, data_size[1]*data_size[2]*data_size[3]))
      labels = Variable(target).to(device)
      if is_SVM:
        labels2 = np.zeros((batch_size, 10))
        for l in range(labels.shape[0]):
          np.add.at(labels2[l], labels[l], 1)
        labels = torch.from_numpy(labels2)

      optimizer.zero_grad()
      outputs = model(images)
      loss = criterion(outputs, labels)
      loss.backward()
      optimizer.step()

    with torch.no_grad():
      correct = 0
      total = 0
      for k, (data, target) in enumerate(test_data):
        data_size = data.size()
        images = Variable(data.view(-1, data_size[1]*data_size[2]*data_size[3]))
        labels = Variable(target).to(device)

        outputs = model(images)
        _, y_pre = torch.max(outputs.data, 1)
        total+= labels.size(0)
        correct+= (y_pre == labels).sum()
      acc_arry.append(100 * correct/total)
  return np.array(acc_arry)

Farklı weightler için regularized ve SGD'li logistic regression fonksiyonu

In [ ]:
def LogReg_Regu(test_loader, train_loader , param_n, data_name, n=10):
  LogReg_Regu_acc = []
  criterion = torch.nn.CrossEntropyLoss()

  for i in range(n):
    model = Logistic_Reg_Model(param_n).to(device)
    optimizer = torch.optim.SGD(model.parameters(),lr=0.01, weight_decay=i*0.1)
    acc = trainer(model, criterion, optimizer, test_loader, train_loader)
    print(acc)
    LogReg_Regu_acc.append(acc)
  return LogReg_Regu_acc


SGD ve momentumlu logistic regression fonksiyonu

In [ ]:
def LogReg_Mom(test_loader, train_loader, param_n, data_name, n=10):
  LogReg_Mom_acc = []
  criterion = torch.nn.CrossEntropyLoss()

  for i in range(n):
    model = Logistic_Reg_Model(param_n).to(device)
    optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=i*0.1)
    acc = trainer(model, criterion, optimizer, test_loader, train_loader)
    print(acc)
    LogReg_Mom_acc.append(acc)
  return LogReg_Mom_acc

Nesterov momentumu ile logistic regression fonksiyonu

In [ ]:
def LogReg_Nes(test_loader, train_loader, param_n, data_name, n=10):
  LogReg_Nes_acc = []
  criterion = torch.nn.CrossEntropyLoss()

  for i in range(1,n):
    model = Logistic_Reg_Model(param_n).to(device)
    optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=i*0.1, nesterov=True)
    acc = trainer(model, criterion, optimizer, test_loader, train_loader)
    print(acc)
    LogReg_Nes_acc.append(acc)
  return LogReg_Nes_acc

Adagrad ile logistic regression fonksiyonu

In [ ]:
def LogReg_Adagrad(test_loader, train_loader, param_n, data_name):
  LogReg_Adagrad_acc = []
  criterion = torch.nn.CrossEntropyLoss()

  model = Logistic_Reg_Model(param_n).to(device)
  optimizer = torch.optim.Adagrad(model.parameters())
  acc = trainer(model, criterion, optimizer, test_loader, train_loader)
  print(acc)
  LogReg_Adagrad_acc.append(acc)
  return LogReg_Adagrad_acc

RMS ile logistic regression fonksiyonu

In [ ]:
def LogReg_RMS(test_loader, train_loader, param_n, data_name):
  LogReg_RMS_acc = []
  criterion = torch.nn.CrossEntropyLoss()

  model = Logistic_Reg_Model(param_n).to(device)
  optimizer = torch.optim.RMSprop(model.parameters())
  acc = trainer(model, criterion, optimizer, test_loader, train_loader)
  print(acc)
  LogReg_RMS_acc.append(acc)
  return LogReg_RMS_acc

Adam optimizayon methodu ile logistic regression fonksiyonu

In [ ]:
def LogReg_Adam(test_loader, train_loader, param_n, data_name):
  LogReg_Adam_acc = []
  criterion = torch.nn.CrossEntropyLoss()

  model = Logistic_Reg_Model(param_n).to(device)
  optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
  acc = trainer(model, criterion, optimizer, test_loader, train_loader)
  print(acc)
  LogReg_Adam_acc.append(acc)
  return LogReg_Adam_acc

SVM fonksiyonu

In [ ]:
def SoftMargSVM(test_loader, train_loader, param_n, data_name, n=10):
  SVM_acc = []
  criterion = torch.nn.MultiLabelSoftMarginLoss()

  for i in range(n):
    model = SVM_Model(param_n).to(device)
    optimizer = torch.optim.SGD(model.parameters(), lr=0.01, weight_decay=i*0.1)
    acc = trainer(model, criterion, optimizer, test_loader, train_loader, is_SVM=True)
    print(acc)
    SVM_acc.append(acc)
  return SVM_acc

MNIST datasetini kullanıma hazır hale getirdik


In [ ]:
train_dataset = datasets.MNIST(root='./data', train=True, transform=transforms.ToTensor(), download=True)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
train_loader2 = torch.utils.data.DataLoader(dataset=train_dataset, shuffle=True)
test_dataset = datasets.MNIST(root='./data', train=False, transform=transforms.ToTensor())
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, shuffle=False)

CIFAR 10 datasetini kullanıma hazır hale getirdik

In [ ]:
train_datasetC = datasets.CIFAR10(root='./data', train=True, transform=transforms.ToTensor(), download=True)
train_loaderC = torch.utils.data.DataLoader(dataset=train_datasetC, batch_size=batch_size, shuffle=True)
train_loader2C = torch.utils.data.DataLoader(dataset=train_datasetC, shuffle=True)
test_datasetC = datasets.CIFAR10(root='./data', train=False, transform=transforms.ToTensor())
test_loaderC = torch.utils.data.DataLoader(dataset=test_datasetC, shuffle=False)

MNIST ve CIFAR10 için regularization ile logistic regression 

In [ ]:
LogReg_Regu_acc = LogReg_Regu(test_loader, train_loader, 784, 'MNIST', n=10)
LogReg_Regu_accM = LogReg_Regu(test_loaderC, train_loaderC, 3072, 'CIFAR10', n=10)

Momentum ile logistic regression

In [ ]:
LogReg_Mom_acc = LogReg_Mom(test_loader, train_loader, 784, 'MNIST', n=10)
LogReg_Mom_accC = LogReg_Mom(test_loaderC, train_loaderC, 3072, 'CIFAR10', n=10)

Nesterov momentumu ile logistic regression

In [ ]:
LogReg_Nes_acc = LogReg_Nes(test_loader, train_loader, 784, 'MNIST', n=10)
LogReg_Nes_accC = LogReg_Nes(test_loaderC, train_loaderC, 3072, 'CIFAR10', n=10)

ADAGRAD ile logistic regression

In [ ]:
LogReg_Adagrad_acc = LogReg_Adagrad(test_loader, train_loader, 784, 'MNIST')
LogReg_Adagrad_accC = LogReg_Adagrad(test_loaderC, train_loaderC, 3072, 'CIFAR10')

RMSprop ile logistic regression

In [ ]:
LogReg_RMS_acc = LogReg_RMS(test_loader, train_loader, 784, 'MNIST')
LogReg_RMS_accC = LogReg_RMS(test_loaderC, train_loaderC, 3072, 'CIFAR10')

Adam ile logistic regression

In [ ]:
LogReg_Adam_acc = LogReg_Adam(test_loader, train_loader, 784, 'MNIST')
LogReg_Adam_accC = LogReg_Adam(test_loaderC, train_loaderC, 3072, 'CIFAR10')

Support vector machine

In [ ]:
SVM_acc = SoftMargSVM(test_loader, train_loader2, 784, 'MNIST', n=10)
SVM_accC = SoftMargSVM(test_loaderC, train_loader2C, 3072, 'CIFAR10', n=10)

Scikit Learn kütüphanesi ile RBF Kerneli kullanılarak SVM

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets
from sklearn import svm , metrics
from sklearn.datasets import fetch_openml
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split

mnist = fetch_openml('mnist_784', cache=False)
X = mnist.data.astype('float32')
y = mnist.target.astype('int64')
X /= 255.0
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)

svc = svm.SVC(C=1000,gamma=0.05)
svc.fit(X_train,y_train)
y_pred = svc.predict(X_test)

svc_withreg = svm.SVC(C=10,gamma=0.05)
svc_withreg.fit(X_train,y_train)
y_pred2 = svc_withreg.predict(X_test)

acc1 = metrics.accuracy_score(y_test, y_pred)
acc2 = metrics.accuracy_score(y_test, y_pred2)
print("SVC accuracy with regularization:",acc2)
print("SVC accuracy without regularization:",acc1)

clf = SGDClassifier(alpha=0.001, max_iter=10000).fit(X_train, y_train)
y_predSGD = clf.predict(X_test)
acc3 = metrics.accuracy_score(y_test, y_predSGD)
print("SVC accuracy using SGD optimizer:",acc3)

#Sonuçların İncelenmesi ve Karşılaştırılması





\begin{array}{|c|c|c|}
\hline 
Method  & MNIST & CIFAR \;10  \\
\hline
\text{LogRegression  without Regularization} & \%90.04 & \%37.24  \\
\hline
\text{LogRegression  with Regularization*} & \%76.12 & \%20.36  \\
\hline
\text{LogRegression  with Momentum**} & \%91.01 & \%35.73  \\
\hline
\text{LogRegression  with Nesterov Momentum} & \%91.32 & \%31.17  \\
\hline
\text{LogRegression  with ADAGRAD} & \%90.59 & \%35.97  \\
\hline
\text{LogRegression  with RMSProp} & \%87.87 & \%10.5  \\
\hline
\text{LogRegression  with ADAM} & \%92.06 & \%23.03  \\
\hline
\text{SVM without regularization} & \%90.94 & \%23.03  \\
\hline
\text{SVM with regularization*} & \%48.9 & \%12.14  \\
\hline
\text{SVM with RBF Kernel (Scikit Learn)} & \%97.02   \\
\hline
\end{array}

                               *Regularization sabiti olarak 0.5 alınmıştır
                               **Momentum sabiti 0.8 olarak 0.5 alınmıştır

